<a href="https://colab.research.google.com/github/firdaaacy/Praktikum-PPM/blob/main/Praktikum%206.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 6. Support Vector Machine (SVM)


### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [1]:
#mengunduh dataset iris dari link yang sudah diberikan
! wget https://dataset-ppm.s3.amazonaws.com/iris.csv

--2021-07-10 12:31:11--  https://dataset-ppm.s3.amazonaws.com/iris.csv
Resolving dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)... 52.216.143.132
Connecting to dataset-ppm.s3.amazonaws.com (dataset-ppm.s3.amazonaws.com)|52.216.143.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4609 (4.5K) [text/csv]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   4.50K  --.-KB/s    in 0s      

2021-07-10 12:31:11 (102 MB/s) - ‘iris.csv’ saved [4609/4609]



Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [2]:
#mengimpor library pandas untuk data analitis
import pandas as pd
#mengimpor library numpy untuk array
import numpy as np
#membaca data iris dengan library pandas dengan method read csv
data = pd.read_csv('iris.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [3]:
#melihat 5 data teratas dengan method .head()
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


Metode Support Vector Machine (SVM) dasar hanya mampu melakukan klasifikasi data yang terdiri dari dua kelas, atau disebut klasifikasi biner. Dataset iris memiliki 3 kelas, sehingga salah satu kelasnya, yaitu Iris-virginica perlu dihapus. Cell berikut menghapus data yang memiliki kelas Iris-virginica

In [4]:
#menghapus kelas iris-virginica agar hanya tersisa 2 kelas dengan perinta drop
data.drop(data[data['species']=='Iris-virginica'].index,inplace=True)

SVM juga memiliki keterbatasan, yaitu hanya mampu menerima kelas dalam bentuk numerik. Cell berikut mengubah kelas menjadi bilangan, Iris setosa menjadi -1 dan Iris-versicolor menjadi 1

In [5]:
#mengganti kelas iris-sentosa menjadi -1 dan iris-versicolor menjadi 1
data['species']=data['species'].map({'Iris-setosa':-1,'Iris-versicolor':1})

Tampilkan beberapa data untuk mengecek hasilnya

In [6]:
#melihat 5 data teratas dengan method .head()
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,-1
1,4.9,3.0,1.4,0.2,-1
2,4.7,3.2,1.3,0.2,-1
3,4.6,3.1,1.5,0.2,-1
4,5.0,3.6,1.4,0.2,-1


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [7]:
#melakukan import sebagian library yang akan digunakan
#pada kasus  ini hanya mengimport train_test_split dari modul sklearn.model_selection
from sklearn.model_selection import train_test_split
#melakukan pembagian dataset dengan pembagian 20% data uji dan 80% data latih
data_latih, data_uji = train_test_split(data, test_size = 0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 80 data, dan **data_uji** terdiri dari 20 data

In [8]:
#menampilkan jumlah baris data uji yang sudah dibagi 20% dari dataset
print(data_uji.shape[0])
#menampilkan jumlah baris data latih yang sudah dibagi 80% dari dataset
print(data_latih.shape[0])

20
80


Pisahkan label/kategori dari data latih dan data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_latih** dan **label_uji**

In [9]:
#melakukan pemisahan antara fitur dan label dengan melakukan pop pada kolom spesies pada label latih
label_latih = data_latih.pop('species')
#melakukan pemisahan antara fitur dan label dengan melakukan pop pada kolom spesies pada label uji
label_uji = data_uji.pop('species')

In [10]:
#melihat 5 data teratas dengan method .head()
data_uji.head()

,sepal_length,sepal_width,petal_length,petal_width
68,6.2,2.2,4.5,1.5
38,4.4,3.0,1.3,0.2
10,5.4,3.7,1.5,0.2
63,6.1,2.9,4.7,1.4
81,5.5,2.4,3.7,1.0


## 3) Proses Training

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [11]:
# menghitung cost gradien menggunakan fungsi 
def hitung_cost_gradient(W,X,Y,regularization):
  #menghitung jarak dengan mengurangkan 1 dengan y * dot product x dan w
  jarak = 1 - (Y* np.dot(X,W))
  #menbuat array 0 seukuran dengan panjang w
  dw = np.zeros(len(W))
  #kondisi dimana jarak < -1
  if max(0,jarak)==0:
    #nilai di maka nilai W
    di= W
  #jika tidak memenuhi kondisi maka kondisi else akan dieksekusi
  else:
    #nilai di akan diset dari W - (regularization*Y*X)
    di = W - (regularization * Y*X)
  #nilai di akan disimpan sebagai intercept
  dw += di
  #mengembalikan nilai w*x + c
  return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD 

In [12]:
# mengimport kelas shuffle dari library sklearn utils
from sklearn.utils import shuffle
#membuat fungsi SGD untuk menghitung bobot
def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  #mengubah data latih yang berbentuk dataframe menjadi array mengunakan numpy
  data_latih = data_latih.to_numpy()
  #mengubah label latih yang berbentuk dataframe menjadi array mengunakan numpy
  label_latih = label_latih.to_numpy()
  #membuat variabel bobot yang berisi aray kosong yang ukurannya sama dengan banyak kolom
  bobot = np.zeros(data_latih.shape[1])
  #membuat perulangan untuk menghitung bobot
  for epoch in range(1,max_epoch):
    #melakukan shuffling nilai X dan Y agar tidak terjadi pola perhitungan bobot yang sama
    X,Y =shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      #menghitung delta dengan fungsi hitung cost gradient
      delta=hitung_cost_gradient(bobot,x,Y[index],regularization)
      #menentukan nilai bobot dengan mengurangi bobot dengan learning rate * delta
      bobot = bobot - (learning_rate * delta)
  #mengembalikan nilai bobot
  return bobot

Proses training dilakukan dengan memanggil fungsi **sgd** dengan parameter input berupa data latih dan label latih. Parameter learning rate dan max epoch menggunakan nilai *default* nya

In [13]:
#menghitung nilai bobot dengan fungsi sgd
W = sgd(data_latih,label_latih)
#mencetak nilai W
print(W)

[-0.26796816 -0.63881552  1.13955217  0.51675419]


## 4) Proses *testing*
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [14]:
#membuat fungsi testing 
def testing(W,data_uji):
  #membuat array kosong yang akan digunakan menyimpan nilai prediksi
  prediksi = np.array([])
  #melakukan perulangan untuk menguji tiap baris data uji
  for i in range(data_uji.shape[0]):
    #melakukan perhitungan dot product antara data uji dengan matriks W
    y_prediksi = np.sign(np.dot(W,data_uji.to_numpy()[i]))
    #menambahkan nilai prediksi ke array prediksi
    prediksi = np.append(prediksi,y_prediksi)
  # mengembalikan nilai prediksi secara keseluruhan
  return prediksi

Lakukan pengujian menggunakan seluruh data uji. Bandingkan hasilnya dengan nilai label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar.

In [15]:
#melakukan prediksi dengan memanggil fungsi testing
y_prediksi = testing(W,data_uji)
#menampilkan jumlah prediksi yang sama dengan label sebenarnya
print(sum(y_prediksi==label_uji))

20


In [16]:
#menampilkan jumlah prediksi yang sama dengan label sebenarnya
prediksiBenar = sum(y_prediksi==label_uji)
#menampilkan jumlah data sebenarnya
jumlahSebenarnya = label_uji.shape[0]
#menampilkan nilai prediksi yang benar
print("Prediksi benar =", prediksiBenar)
# menampilkan jumlah data sebenarnya
print("Jumlah label sebenarnya =", jumlahSebenarnya)
# menghitung akurasi dengan membandingkan nilai benar dengan jumlah 
print("Akurasi = ", (prediksiBenar/jumlahSebenarnya))

Prediksi benar = 20
Jumlah label sebenarnya = 20
Akurasi =  1.0


## TUGAS
Pada tugas kali ini Anda masih melakukan klasifikasi dataset iris, tetapi menggunakan kelas Iris-versicolor dan Iris-virginica. Apakah hasilnya lebih baik jika dibandingkan dengan Iris-setosa dan Iris-versicolor?

In [17]:
#membaca ulang data iris dengan library pandas dengan method read csv
data1 = pd.read_csv('iris.csv')

In [18]:
#menghapus kelas iris-setosa agar hanya tersisa 2 kelas dengan perinta drop
data1.drop(data1[data1['species']=='Iris-setosa'].index,inplace=True)

In [19]:
#melihat 5 data teratas dengan method .head()
data1.head()

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,Iris-versicolor
51,6.4,3.2,4.5,1.5,Iris-versicolor
52,6.9,3.1,4.9,1.5,Iris-versicolor
53,5.5,2.3,4.0,1.3,Iris-versicolor
54,6.5,2.8,4.6,1.5,Iris-versicolor


In [20]:
#mengganti kelas iris-versicolor menjadi -1 dan iris-virginica menjadi 1
data1['species']=data1['species'].map({'Iris-versicolor':-1,'Iris-virginica':1})

In [21]:
#melihat 5 data teratas dengan method .head()
data1.head()

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,-1
51,6.4,3.2,4.5,1.5,-1
52,6.9,3.1,4.9,1.5,-1
53,5.5,2.3,4.0,1.3,-1
54,6.5,2.8,4.6,1.5,-1


In [22]:
#melakukan import sebagian library yang akan digunakan
#pada kasus  ini hanya mengimport train_test_split dari modul sklearn.model_selection
from sklearn.model_selection import train_test_split
#melakukan pembagian dataset dengan pembagian 20% data uji dan 80% data latih
data_latih_tugas, data_uji_tugas = train_test_split(data1, test_size = 0.2)

In [23]:
#melakukan pemisahan antara fitur dan label dengan melakukan pop pada kolom spesies pada label latih
label_latih_tugas = data_latih_tugas.pop('species')
#melakukan pemisahan antara fitur dan label dengan melakukan pop pada kolom spesies pada label uji
label_uji_tugas = data_uji_tugas.pop('species')

In [24]:
#melihat 5 data teratas dengan method .head()
data1.head()

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,-1
51,6.4,3.2,4.5,1.5,-1
52,6.9,3.1,4.9,1.5,-1
53,5.5,2.3,4.0,1.3,-1
54,6.5,2.8,4.6,1.5,-1


In [25]:
#menghitung nilai bobot dengan fungsi sgd
W = sgd(data_latih_tugas,label_latih_tugas)
#mencetak nilai W
print(W)

[-4.93668993 -3.75837246  6.17726447  6.93407883]


In [26]:
#melakukan prediksi dengan memanggil fungsi testing
y_prediksi_tugas = testing(W,data_uji_tugas)
#menampilkan jumlah prediksi yang sama dengan label sebenarnya
print(sum(y_prediksi_tugas ==label_uji_tugas))

18


In [27]:
#menampilkan jumlah prediksi yang sama dengan label sebenarnya
prediksiBenar = sum(y_prediksi_tugas ==label_uji_tugas)
#menampilkan jumlah data sebenarnya
jumlahSebenarnya = label_uji_tugas.shape[0]
#menampilkan nilai prediksi yang benar
print("Prediksi benar =", prediksiBenar)
# menampilkan jumlah data sebenarnya
print("Jumlah label sebenarnya =", jumlahSebenarnya)
# menghitung akurasi dengan membandingkan nilai benar dengan jumlah 
print("Akurasi = ", (prediksiBenar/jumlahSebenarnya))

Prediksi benar = 18
Jumlah label sebenarnya = 20
Akurasi =  0.9




> Hasilnya tidak lebih baik karena hanya mampu mengenali 19 diantara 20 data uji (pada kelas Iris-versicolor dan Iris-virginica), sedangkan pada Iris-versicolor dan Iris-setosa mampu memprediksi 20 label dengan benar dari 20 data uji. Hal ini berarti antara kelas Iris-versicolor dan Iris-setosa hyperplane yang dibuat benar-benar memisahkan 2 kelas tersebut dan lebih baik jika dibandingkan dengan hyperplane pada Iris-versicolor dan Iris-virginica.


Lakukan proses training ulang, tetapi menggunakan nilai max_epoch sebesar 5000. Anda dapat memberikan parameter max_epoch=5000 saat memanggil fungsi **sgd**. Apakah hasilnya lebih baik? Mengapa?

In [28]:
# training max_epoch = 5000
W_tugas_maxepoch = sgd(data_latih_tugas,label_latih_tugas, max_epoch=5000)
print(W_tugas_maxepoch)

[-5.9472972  -5.06400878  7.80597381  8.4400674 ]


In [29]:
#melakukan prediksi dengan memanggil fungsi testing
y_prediksi_tugas_epoch  = testing(W_tugas_maxepoch ,data_uji_tugas)
#menampilkan jumlah prediksi yang sama dengan label sebenarnya
print(sum(y_prediksi_tugas_epoch ==label_uji_tugas))

18


>Setelah melakukan proses training ulang menggunakan nilai max_epoch = 5000, hasil yang didapatkan sama dengan pengujian dengan nilai epoch 1000 yakni 19 prediksi benar dari 20 data uji. Max epoch merupakan batas perulangan untuk mencari bobot/ cost function terkecil. Jika nilai cost function semakin minimal maka persamaan garis yang dihasilkan semakin konvergen sehingga hyperplanenya optimal. Jika max_epoch semakin besar maka nilai yang didapatkan akan semakin konvergen, namun nilai ini memiliki batas minimum. Ketika sudah mencapai batas minimum maka nilainya akan tetap (tidak bisa lebih kecil lagi).

Terakhir, lakukanlah seleksi fitur sederhana dengan mengambil hanya fitur sepal length dan sepal width saja. Apakah hasilnya lebih baik?

In [30]:
# mengambil fitur sepal_length dan sepal_width dari data_latih_tugas
data_latih_t3 = data_latih_tugas[['sepal_length', 'sepal_width']]
# mengambil fitur sepal_length dan sepal_width dari data_uji_tugas
data_uji_t3 = data_uji_tugas[['sepal_length', 'sepal_width']]

In [31]:
#cek jumlah kolom pada data latih
print(data_latih_t3.shape[1])

2


In [32]:
#cek jumlah kolom pada data uji
print(data_uji_t3.shape[1])

2


In [33]:
#melihat 5 data teratas dengan method .head()
data_latih_t3.head()

,sepal_length,sepal_width
95,5.7,3.0
59,5.2,2.7
130,7.4,2.8
58,6.6,2.9
78,6.0,2.9


In [34]:
#melihat 5 data teratas dengan method .head()
data_uji_t3.head()

,sepal_length,sepal_width
147,6.5,3.0
56,6.3,3.3
50,7.0,3.2
101,5.8,2.7
63,6.1,2.9


In [35]:
data_latih_t3.head()

,sepal_length,sepal_width
95,5.7,3.0
59,5.2,2.7
130,7.4,2.8
58,6.6,2.9
78,6.0,2.9


In [36]:
#menghitung nilai bobot dengan fungsi sgd
W_t3 = sgd(data_latih_t3,label_latih_tugas)
#mencetak nilai W
print(W_t3)

[ 0.21675386 -0.81165132]


In [37]:
#melakukan prediksi dengan memanggil fungsi testing
y_prediksi_t3= testing(W_t3,data_uji_t3)
#menampilkan jumlah prediksi yang sama dengan label sebenarnya
print(sum(y_prediksi_t3 ==label_uji_tugas))

10


In [38]:
#menampilkan jumlah prediksi yang sama dengan label sebenarnya
prediksiBenar = sum(y_prediksi_t3 ==label_uji_tugas)
#menampilkan jumlah data sebenarnya
jumlahSebenarnya = label_uji_tugas.shape[0]
#menampilkan nilai prediksi yang benar
print("Prediksi benar =", prediksiBenar)
# menampilkan jumlah data sebenarnya
print("Jumlah label sebenarnya =", jumlahSebenarnya)
# menghitung akurasi dengan membandingkan nilai benar dengan jumlah 
print("Akurasi = ", (prediksiBenar/jumlahSebenarnya))

Prediksi benar = 10
Jumlah label sebenarnya = 20
Akurasi =  0.5


In [39]:
# training max_epoch = 5000
W_t3_maxepoch = sgd(data_latih_t3,label_latih_tugas, max_epoch=5000)
#mencetak nilai W
print(W_t3_maxepoch)

[ 0.22057804 -0.81449865]


In [40]:
#melakukan prediksi dengan memanggil fungsi testing
y_prediksi_t3_epoch= testing(W_t3_maxepoch,data_uji_t3)
#menampilkan jumlah prediksi yang sama dengan label sebenarnya
print(sum(y_prediksi_t3 ==label_uji_tugas))

10


In [41]:
#menampilkan jumlah prediksi yang sama dengan label sebenarnya
prediksiBenar = sum(y_prediksi_t3_epoch==label_uji_tugas)
#menampilkan jumlah data sebenarnya
jumlahSebenarnya = label_uji_tugas.shape[0]
#menampilkan nilai prediksi yang benar
print("Prediksi benar =", prediksiBenar)
# menampilkan jumlah data sebenarnya
print("Jumlah label sebenarnya =", jumlahSebenarnya)
# menghitung akurasi dengan membandingkan nilai benar dengan jumlah 
print("Akurasi = ", (prediksiBenar/jumlahSebenarnya))

Prediksi benar = 10
Jumlah label sebenarnya = 20
Akurasi =  0.5


>Hasil prediksi pada data yang dilakukan seleksi fitur tidak lebih baik dari data yang menggunakan 4 fitur (tanpa seleksi fitur) dengan akurasi hanya 0.65 (13 data yang benar) saja dari 20 data uji. Hal ini dikarenakan fitur petal (length dan width) memiliki peran yang penting dalam menentukan label dari data, maka jika hanya menggunakan fitur sepal (length dan width) saja hasil prediksi yang dihasilkan kurang maksimal.